In [1]:
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn

from sklearn.datasets import load_breast_cancer
# import matplotlib.pyplot as plt

In [2]:
data = load_breast_cancer()

In [3]:
X = data.data
y = data.target

In [8]:
class ToTensor:

    def __call__(self, sample):
        inputs, labels = sample
        return torch.from_numpy(inputs), torch.from_numpy(labels)

class CancerDataset(Dataset):
    
    def __init__(self, X:np.array, y:np.array, transform=None):
        """
        Args:
            X: Features.
            y: target.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)
        self.n_samples = self.X.shape[0]
        self.transform = transform

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        
        sample = self.X[idx], self.y[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample

train_dataset = CancerDataset(X,y)#, transform=ToTensor())

In [ ]:
# device config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using: {device}")
input_size = X.shape[0]  # qtd_features

# hyper parameters

hl_1 = 12
# hl_2 = 10
hl_3 = 7
# hl_4 = 5
hl_5 = 4
# hl_6 = 3
# hl_7 = 2

hls = [hl_1,hl_3,hl_5]

num_classes = 2
num_epochs = 15
batch_size = 50
learning_rate = 0.001


class MLP(nn.Module):

    def __init__(self, input_size, h1, h3, h5, num_classes):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(input_size, h1)
        self.l2 = nn.Linear(h1, h3)
        self.l3 = nn.Linear(h3, h5)
        self.l4 = nn.Linear(h5, num_classes)
        self.act = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.act(out)
        out = self.l2(out)
        out = self.act(out)
        out = self.l3(out)
        out = self.act(out)
        out = self.l4(out)
        return out


model = MLP(input_size, hl_1, hl_3, hl_5, num_classes).to(device)

NameError: name 'torch' is not defined

In [10]:
# This will apply the softmax
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size)

n_total_steps = len(train_loader)

In [11]:
for i, (X, y) in enumerate(train_loader):
    print(X.shape, y.shape)
    break

torch.Size([50, 30]) torch.Size([50])


In [12]:
for epoch in range(num_epochs):
    for i, (X, y) in enumerate(train_loader):
        features = X.float().to(device)
        labels = y.float().to(device)

        # forward
        outputs = model(features.float())
        loss = criterion(outputs, labels)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"epoch {epoch + 1} / {num_epochs}, step {i + 1}/{n_total_steps}, loss = {loss.item():.4f}")

RuntimeError: CUDA error: CUBLAS_STATUS_INVALID_VALUE when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`

In [ ]:
with torch.no_grad():
    model.eval()
    n_correct = 0
    n_samples = 0

    for images, labels in test_loader:
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predictions = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()

    acc = 100 * n_correct / n_samples
    print(f"Accuracy: {acc:.2f}")

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
# import matplotlib.pyplot as plt

# device config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = 784  # 28x28

# hyper parameters

hidden_size = 100
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transforms.ToTensor())
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, shuffle=False, batch_size=batch_size)


class MLP(nn.Module):

    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out


model = MLP(input_size, hidden_size, num_classes).to(device)

# This will apply the softmax
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)
        print(images.shape, labels.shape)
        

        # forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"epoch {epoch + 1} / {num_epochs}, step {i + 1}/{n_total_steps}, loss = {loss.item():.4f}")


with torch.no_grad():
    model.eval()
    n_correct = 0
    n_samples = 0

    for images, labels in test_loader:
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predictions = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()

    acc = 100 * n_correct / n_samples
    print(f"Accuracy: {acc:.2f}")
